# Set up

Load libraries

In [1]:
import os
import sys, getopt
from pathlib import Path
import xml.etree.cElementTree as ET
import pandas as pd

In [2]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs

In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# Read inventory 
Read gsheet document with updated inventory

In [4]:
def read_gsheet(SPREADSHEET_ID,RANGE_NAME, JSONFILE):
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(JSONFILE, SCOPES) 
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')


In [5]:
conf_json = Path(os.path.expanduser('~')) / '.secrets/googlecloud/python-gsheet-api.json'
my_doc='105V_zTZ-Hzj1ZuZDt1HFo750viegjmM2zyfPvJmPDF4'
read_gsheet(my_doc,'Assessment Target!A1:AA10000',conf_json)

In [6]:
inventory=pd.DataFrame(values_input[1:], columns=values_input[0])
inventory['Assessment ID']=inventory['Ecosystem ID'].str.replace('_[0-9]+$', '', regex=True)

# Database

In [8]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm' # we need admin privileges to make changes

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

We need admin privileges to make these changes

In [9]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
#qry = """UPDATE rle.assessments set asm_id=%s where asm_id=%s"""
#cur.execute(qry, ('Schaefer_RockyShores_NSW_2020','Schaefer_Rocky_Intertidal_Australia'))
qry = """INSERT INTO ref_list(ref_code,ref_cite) VALUES (%s,%s) ON CONFLICT DO NOTHING"""
cur.execute(qry, ('Roche_EasterScotianShelf_2020','Roche EasterScotianShelf 2020 ???'))
cur.execute(qry, ('Javed_RLE_AbuDhabi_2021','Javed_RLE_AbuDhabi_2021 ???'))
cur.execute(qry, ('Carre_RLE_Madagascar_2020','Carre_RLE_Madagascar_2020 ???'))
cur.execute(qry, ('Rouget_SouthAfrica_2004',' ???'))
cur.execute(qry, ('Obura_etal_CoralReef_WIO_2021',' ???'))
cur.execute(qry, ('Mases-Garcia_OaxacaForests_2021',' ???'))
cur.execute(qry, ('Betancourt_ForestsUruguay_2021',' ???'))
qry = """INSERT INTO rle.assessments(ref_code,asm_id) VALUES (%s,%s) ON CONFLICT DO NOTHING"""
cur.execute(qry, ('Roche_EasterScotianShelf_2020','Roche_EasterScotianShelf_2020'))
cur.execute(qry, ('Javed_RLE_AbuDhabi_2021','Javed_RLE_AbuDhabi_2021'))
cur.execute(qry, ('Carre_RLE_Madagascar_2020','Carre_RLE_Madagascar_2020'))
cur.execute(qry, ('Rouget_SouthAfrica_2004','Rouget_SouthAfrica_2004'))
cur.execute(qry, ('Obura_etal_CoralReef_WIO_2021','Obura_etal_CoralReef_WIO_2021'))
cur.execute(qry, ('Mases-Garcia_OaxacaForests_2021','Mases-Garcia_OaxacaForests_2021'))
cur.execute(qry, ('Betancourt_ForestsUruguay_2021','Betancourt_ForestsUruguay_2021'))

cur.close()
conn.commit()
if conn is not None:
    conn.close()

In [25]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
qry = """
SELECT asm_id,eco_id,eco_name,eco_name_orig, eco_name_lang,external_code,assessment_date,
overall_risk_category,risk_category_bounds
FROM rle.assessment_units u
JOIN rle.assessment_overall USING(eco_id)
"""
cur.execute(qry)
assm_units = cur.fetchall()
assm_units_df=pd.DataFrame(assm_units,
                columns=['asm_id','eco_id','eco_name','eco_name_orig', 'eco_name_lang','external_code','assessment_date',
'overall_risk_category','risk_category_bounds'])

conn.commit()
if conn is not None:
    conn.close()


,asm_id,eco_id,eco_name,eco_name_orig,eco_name_lang,external_code,assessment_date,overall_risk_category,risk_category_bounds
0,Carre_RLE_France_2012,Carre_RLE_France_2012:1,Le bassin d’Arcachon,Le bassin d’Arcachon,en,Carre_RLE_France_2012:1,2012-01-01,None,None
1,Carre_RLE_France_2012,Carre_RLE_France_2012:2,Les lacs et étangs du littoral aquitain,Les lacs et étangs du littoral aquitain,en,Carre_RLE_France_2012:2,2012-01-01,None,None
2,Carre_RLE_France_2012,Carre_RLE_France_2012:3,Les lagunes méditerranéennes françaises,Les lagunes méditerranéennes françaises,en,Carre_RLE_France_2012:3,2012-01-01,None,None
3,Carre_RLE_France_2012,Carre_RLE_France_2012:4,Les marais maritimes atlantiques,Les marais maritimes atlantiques,en,Carre_RLE_France_2012:4,2012-01-01,None,None
4,Holdaway_Uncommon_NewZealand_2012,Holdaway_Uncommon_NewZealand_2012:1,Active sand dune,Active sand dune,en,Holdaway_Uncommon_NewZealand_2012:1,2012-01-01,None,None
...,...,...,...,...,...,...,...,...,...
2179,Kontula_FinlandHabitat_2018,RLE-FI-T11.06,T11.06 Mountain acidic and intermediatebasic b...,None,en,RLE-FI-T11.06,2018-10-01,LC,None
2180,Kontula_FinlandHabitat_2018,RLE-FI-T11.07.01,T11.07.01 Acidic and intermediate-basic talus ...,None,en,RLE-FI-T11.07.01,2018-10-01,LC,None
2181,Kontula_FinlandHabitat_2018,RLE-FI-T11.07.02,T11.07.02 Calcareous talus formations,None,en,RLE-FI-T11.07.02,2018-10-01,LC,None
2182,Kontula_FinlandHabitat_2018,RLE-FI-T12.01,T12.01 Canyons,None,en,RLE-FI-T12.01,2018-10-01,LC,None


In [28]:
inv_ss=inventory[inventory['Reference label']=='Keith_Foundations_2013'][['Ecosystem ID','Assessment ID','AT-Name']]

assm_units_df[assm_units_df.asm_id.isin(inv_ss['Assessment ID'])]

,asm_id,eco_id,eco_name,eco_name_orig,eco_name_lang,external_code,assessment_date,overall_risk_category,risk_category_bounds
163,Essl_GermanTamarisk_2013,Essl_CS_GermanTamarisk_2013:1,German tamarisk pioneer vegetation,German tamarisk pioneer vegetation,en,Essl_CS_GermanTamarisk_2013:1,2013-01-01,EN,None
165,Benson_VineThicket_2013,Benson_CS_VineThicketSEAustralia_2013:1,Mock Olive Wilga Peach Bush Carissa dry sub-tr...,Mock Olive Wilga Peach Bush Carissa dry sub-tr...,en,Benson_CS_VineThicketSEAustralia_2013:1,2013-01-01,EN,None
166,Bonifacio_KRSWetland_2013,Bonifacio_CS_KRSWetland_2013:1,Karst rising springs,Karst rising springs,en,Bonifacio_CS_KRSWetland_2013:1,2013-01-01,CR,None
167,Bonifacio_SeagrassCommunity_2013,Bonifacio_CS_SeagrassCommunity_2013:1,Seagrass meadows of South Australia,Seagrass meadows of South Australia,en,Bonifacio_CS_SeagrassCommunity_2013:1,2013-01-01,EN,EN-CR
170,FaberLangendoen_GreatLakesAlvar_2013,FaberLnagendoen_CS_GreatLakesAlvar_2013:1,Great Lakes Alvar,Great Lakes Alvar,en,FaberLnagendoen_CS_GreatLakesAlvar_2013:1,2013-01-01,EN,VU-EN
171,Holdaway_GraniteGravelFields_2013,Holdaway_CS_GraniteGravelFields_2013:1,Granite gravel fields & sandplains,Granite gravel fields & sandplains,en,Holdaway_CS_GraniteGravelFields_2013:1,2013-01-01,LC,None
172,Keith_AralSea_2013,Keith_CS_AralSea_2013:1,Aral Sea,Aral Sea,en,Keith_CS_AralSea_2013:1,2013-01-01,CO,None
173,Keith_GonakierForest_2013,Keith_CS_GonakierForest_2013:1,Gonakier forests of Senegal,Gonakier forests of Senegal,en,Keith_CS_GonakierForest_2013:1,2013-01-01,CR,None
174,Riecken_RaisedBogs_2013,Riecken_CS_RaisedBogs_2013:1,Raised bogs,Raised bogs,en,Riecken_CS_RaisedBogs_2013:1,2013-01-01,CR,None
175,Keith_GiantKelp_2013,Keith_CS_GiantKelp_2013:1,Giant kelp forests,Giant kelp forests,en,Keith_CS_GiantKelp_2013:1,2013-01-01,EN,EN-CR


In [63]:
# 'Shapiro_CongoBasin_2020', 'Roche_EasterScotianShelf_2020'

inventory[inventory['Reference label']=='Kontula_FinlandHabitat_2018'][['Ecosystem ID','Assessment ID','AT-Name','Overall_RC']]



,Ecosystem ID,Assessment ID,AT-Name,Overall_RC
2982,Kontula_FinlandHabitat_2018_1,Kontula_FinlandHabitat_2018,Pending to process,CR
2983,Kontula_FinlandHabitat_2018_2,Kontula_FinlandHabitat_2018,Pending to process,CR
2984,Kontula_FinlandHabitat_2018_3,Kontula_FinlandHabitat_2018,Pending to process,CR
2985,Kontula_FinlandHabitat_2018_4,Kontula_FinlandHabitat_2018,Pending to process,CR
2986,Kontula_FinlandHabitat_2018_5,Kontula_FinlandHabitat_2018,Pending to process,CR
...,...,...,...,...
3365,Kontula_FinlandHabitat_2018_384,Kontula_FinlandHabitat_2018,Pending to process,DD
3366,Kontula_FinlandHabitat_2018_385,Kontula_FinlandHabitat_2018,Pending to process,DD
3367,Kontula_FinlandHabitat_2018_386,Kontula_FinlandHabitat_2018,Pending to process,DD
3368,Kontula_FinlandHabitat_2018_387,Kontula_FinlandHabitat_2018,Pending to process,DD


In [58]:
ins1 = """
INSERT INTO rle.assessment_units (eco_id,eco_name,eco_name_orig, eco_name_lang,external_code)
VALUES %s
ON CONFLICT DO NOTHING
"""
ins2="""
INSERT INTO rle.assessment_overall (asm_id,eco_id,assessment_date,overall_risk_category)
VALUES %s
ON CONFLICT DO NOTHING
"""

ref='Betancourt_ForestsUruguay_2021'
ref='Obura_etal_CoralReef_WIO_2021'
ref='Mases-Garcia_OaxacaForests_2021'
ref='Javed_RLE_AbuDhabi_2021'
inv_ss = inventory[inventory['Reference label']==ref][['Ecosystem ID','Assessment ID','AT-Name','Overall_RC']]


conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
for ss in inv_ss.to_dict(orient='records'):
    cur.execute(ins1,((ss['Ecosystem ID'],ss['AT-Name'],ss['AT-Name'],'sp',ss['Ecosystem ID']),)) # we will need to fix language later
    cur.execute(ins2,((ss['Assessment ID'],ss['Ecosystem ID'],'2021-01-01',ss['Overall_RC']),))
cur.close()
conn.commit()
if conn is not None:
    conn.close()

In [59]:
(ss['Ecosystem ID'],ss['AT-Name'],ss['AT-Name'],'sp',ss['Ecosystem ID'])


('Carre_RLE_Madagascar_2020_36',
 'Le système lagunaire du littoral de Madagascar',
 'Le système lagunaire du littoral de Madagascar',
 'sp',
 'Carre_RLE_Madagascar_2020_36')

## Coral reefs of the Western Indian Ocean

Update information for the assessments of Coral Reefs from Obura et al.

In [ ]:
ins="UPDATE rle.assessments set countries='{TZ,KE,MZ,KG,YT,MG,SC,MU,RE,ZA}' WHERE asm_id='Obura_etal_CoralReef_WIO_2021'"


In [22]:
records=[
{'eco_id':'Obura_etal_CoralReef_WIO_2021_1', 'countries':['TZ','KE']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_2', 'countries':['TZ','MZ']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_3', 'countries':['KM','YT']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_4', 'countries':['MG']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_5', 'countries':['MG']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_6', 'countries':['SC']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_7', 'countries':['SC']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_8', 'countries':['MU','RE']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_9', 'countries':['MG']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_10', 'countries':['MG']},
{'eco_id':'Obura_etal_CoralReef_WIO_2021_11', 'countries':['ZA','MZ']}]

upd = "update rle.assessment_units set countries=%s where eco_id=%s"
ins = "INSERT INTO rle.assessment_get_xwalk  (eco_id,efg_code,level,membership,assigned_by) values (%s,'M1.3',4,1.0,'Assessment authors') ON CONFLICT DO NOTHING"
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
for record in records:
    cur.execute(upd,((record['countries']),record['eco_id'],))
    cur.execute(ins,(record['eco_id'],))
cur.close()
conn.commit()
if conn is not None:
    conn.close()

In [17]:
record['countries']

['TZ', 'KE']

## Forest of the Congo Basin

In the appendix of the publication there is a table with following information to translate the codes for the forest units:

In [65]:
#X0000
Broad={1:'Dense evergreen %s %s %s rainforest',
       2:'Evergreen and semi-deciduous %s %s %s rainforest',
       3:'Semi-deciduous %s %s %s rainforest',
       4:'Semi-deciduous %s %s %s rainforest with pioneers',
       5:'Marantaceae',
       6:'%s Swamp forest',
       7:'Mangrove',
       8:'Open %s %s %s forest'}
#X000
Climate={1:'northern equatorial',2:'southern equatorial',3:'central'}
#X00
Elevation={1:'lowland',2:'submontane',3:'montane'}
#X0
Biogeographical={1:'northern',
                 2:'northern eastern',
                 3:'southern',
                 4:'north western',
                 5:'eastern',
                 6:'western',}
Flooding={1:'Irregularly Flooded',
          2:'Seasonal Short-Lasting Flood Pulse',
          3:'Stable Water Level',
          4:'Seasonal Flood Pulse',
          5:'Palm-Dominated Seasonal Short Lasting Flood Pulse',
          6:'Palm-Dominated Stable Water Level',
          7:'Palm-Dominated Seasonal Flood Pulse'
}


We will use this code to create a name for the unit, assign a tentative level 3 unit of the typology,  and approximate the countries for the unit:

In [66]:
def decode(record):
    code=record['code']
    if int(code[0]) in (1,2,3,4,8):
        name=Broad[int(code[0])]  % (Climate[int(code[1])], Elevation[int(code[2])], Biogeographical[int(code[3])])
    elif int(code[0]) in (6,):
        name=Broad[int(code[0])] % Flooding[int(code[4])]
    else:
        name=Broad[int(code[0])]

    if int(code[0]) in (1,2,3,4,5):
        if int(code[2]) in (3,):
            efg_code='T1.3'
        else:
            efg_code='T1.1'
    elif int(code[0]) in (6,):
        efg_code='TF1.1'
    elif int(code[0]) in (7,):
        efg_code='MFT1.2'
    elif int(code[0]) in (8,):
        efg_code='T4.2'

    if int(code[3]) in (1,):
        countries=['CF','CM']
    elif int(code[3]) in (2,):
        countries=['CF','CD']
    elif int(code[3]) in (3,5):
        countries=['CD']
    elif int(code[3]) in (4,):
        countries=['CM']
    elif int(code[3]) in (6,):
        countries=['GA','CG','GQ','AO']
    record['name']=name
    record['efg_code']=efg_code
    record['countries']=countries
    return(record)

decode({'code':'63155'})



{'code': '63155',
 'name': 'Palm-Dominated Seasonal Short Lasting Flood Pulse Swamp forest',
 'efg_code': 'TF1.1',
 'countries': ['CD']}

Now, from the table in the manuscript we can link codes to the overall category of risk:

In [67]:

# Dense evergreen
records=[{'code':'11320', 'category':'CR'},
{'code':'11340', 'category':'VU'},
{'code':'12120', 'category':'VU'},
{'code':'12130', 'category':'VU'},
{'code':'12140', 'category':'LC'},
{'code':'12220', 'category':'LC'},
{'code':'12240', 'category':'VU'},
{'code':'12320', 'category':'LC'},
{'code':'12340', 'category':'EN'},
# Evergreen and semi-deciduous
{'code':'22120', 'category':'LC'},
{'code':'22130', 'category':'LC'},
{'code':'22140', 'category':'LC'},
{'code':'22220', 'category':'LC'},
{'code':'22240', 'category':'LC'},
{'code':'22320', 'category':'VU'},
{'code':'22340', 'category':'EN'},
# Semi-Deciduous
{'code':'31110', 'category':'LC'},
{'code':'31120', 'category':'LC'},
{'code':'31130', 'category':'LC'},
{'code':'31140', 'category':'LC'},
{'code':'31220', 'category':'EN'},
{'code':'31240', 'category':'LC'},
{'code':'31320', 'category':'CR'},
{'code':'31340', 'category':'VU'},
{'code':'32120', 'category':'LC'},
{'code':'32130', 'category':'LC'},
{'code':'32140', 'category':'LC'},
{'code':'32220', 'category':'LC'},
{'code':'32240', 'category':'VU'},
{'code':'32320', 'category':'VU'},
{'code':'32340', 'category':'EN'},
# Semi-Deciduous with Pioneer
{'code':'42120', 'category':'VU'},
{'code':'42130', 'category':'LC'},
{'code':'42140', 'category':'VU'},
{'code':'42220', 'category':'EN'},
{'code':'42230', 'category':'CR'},
{'code':'42240', 'category':'VU'},
{'code':'42320', 'category':'CR'},
{'code':'42340', 'category':'EN'},
# Swamp Forest
{'code':'63151', 'category':'VU'},
{'code':'63152', 'category':'LC'},
{'code':'63153', 'category':'LC'},
{'code':'63154', 'category':'LC'},
{'code':'63155', 'category':'LC'},
{'code':'63156', 'category':'LC'},
{'code':'63157', 'category':'LC'},
{'code':'63161', 'category':'VU'},
{'code':'63162', 'category':'LC'},
{'code':'63163', 'category':'LC'},
{'code':'63164', 'category':'VU'},
{'code':'63165', 'category':'LC'},
{'code':'63166', 'category':'LC'},
{'code':'63167', 'category':'LC'},
# Open Forest
{'code':'81110', 'category':'EN'},
{'code':'81120', 'category':'EN'},
{'code':'81140', 'category':'EN'},
{'code':'81240', 'category':'EN'},
{'code':'81320', 'category':'EN'},
{'code':'81340', 'category':'EN'},
{'code':'82130', 'category':'EN'},
{'code':'82230', 'category':'EN'},
# Mangrove'},
{'code':'71140', 'category':'LC'},
# Marantaceae
{'code':'51140', 'category':'LC'}]

Run the function for each record:

In [68]:
for record in records:
    decode(record)

The records are now updated with all information we could extract automatically. Now we can perform three insert operations into the database for each record: one to insert the assessment unit with  name and countries, one to insert the category of risk and link it to an assessment, and one for the crosswalk with the typology:

In [69]:
ins1 = """INSERT INTO  rle.assessment_units 
(eco_id,eco_name,countries) values %s ON CONFLICT DO NOTHING"""
ins2 = """INSERT INTO  rle.assessment_overall 
(eco_id,asm_id,assessment_date,overall_risk_category) values %s ON CONFLICT DO NOTHING"""
ins3 = """INSERT INTO rle.assessment_get_xwalk  
(eco_id,efg_code,level,membership,assigned_by) values %s ON CONFLICT DO NOTHING"""


conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
for record in records:
    cur.execute(ins1, (('Shapiro_CongoBasin_2020_'+record['code'],record['name'],record['countries']),))
    cur.execute(ins2, (('Shapiro_CongoBasin_2020_'+record['code'],'Shapiro_CongoBasin_2020','2020-01-01',record['category']),))
    cur.execute(ins3, (('Shapiro_CongoBasin_2020_'+record['code'],record['efg_code'],5,0.1,'JRFeP'),))

cur.close()
conn.commit()
if conn is not None:
    conn.close()

## Madagascar

Using this quick template to add the Placeholder for the Madagascar assessment:

In [ ]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
qry = """INSERT INTO ref_list(ref_code,ref_cite) VALUES (%s,%s) ON CONFLICT DO NOTHING"""
cur.execute(qry, ('Carre_RLE_Madagascar_2020','Carre_RLE_Madagascar_2020 ???'))
qry = """INSERT INTO rle.assessments(ref_code,asm_id) VALUES (%s,%s) ON CONFLICT DO NOTHING"""
cur.execute(qry, ('Carre_RLE_Madagascar_2020','Carre_RLE_Madagascar_2020'))

cur.close()
conn.commit()
if conn is not None:
    conn.close()

In [ ]:
We then use a subset of the inventory table to add the assessment units to the database:

In [ ]:
ins1 = """
INSERT INTO rle.assessment_units (eco_id,eco_name,eco_name_orig, eco_name_lang,external_code)
VALUES %s
ON CONFLICT DO NOTHING
"""
ins2="""
INSERT INTO rle.assessment_overall (asm_id,eco_id,assessment_date,overall_risk_category)
VALUES %s
ON CONFLICT DO NOTHING
"""

ref='Carre_RLE_Madagascar_2020'
inv_ss = inventory[inventory['Reference label']==ref][['Ecosystem ID','Assessment ID','AT-Name','Overall_RC']]


conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)
for ss in inv_ss.to_dict(orient='records'):
    cur.execute(ins1,((ss['Ecosystem ID'],ss['AT-Name'],ss['AT-Name'],'sp',ss['Ecosystem ID']),)) # we will need to fix language later
    cur.execute(ins2,((ss['Assessment ID'],ss['Ecosystem ID'],'2021-01-01',ss['Overall_RC']),))
cur.close()
conn.commit()
if conn is not None:
    conn.close()

Now we need to add the countries to both units and assessment:

In [70]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)

upd ="""UPDATE rle.assessments set assessment_protocol_code='IUCN RLE v2.2',
    risk_category_code='IUCN RLE', countries='{MG}' 
    WHERE asm_id='Carre_RLE_Madagascar_2020'"""
cur.execute(upd) 

upd ="""UPDATE rle.assessment_units set  countries='{MG}' 
    WHERE eco_id like 'Carre_RLE_Madagascar_2020%'"""
cur.execute(upd) 

cur.close()
conn.commit()
if conn is not None:
    conn.close()

We can look at the codes of the units and propose a cross walk:

In [71]:
ref='Carre_RLE_Madagascar_2020'
inv_ss = inventory[inventory['Reference label']==ref][['Ecosystem ID','Assessment ID','AT-Name','Overall_RC']]


In [72]:
inv_ss

,Ecosystem ID,Assessment ID,AT-Name,Overall_RC
1806,Carre_RLE_Madagascar_2020_1,Carre_RLE_Madagascar_2020,Forêts littorales de L’Est,EN
1807,Carre_RLE_Madagascar_2020_2,Carre_RLE_Madagascar_2020,Forêts denses humides de basse altitude de l’Est,VU
1808,Carre_RLE_Madagascar_2020_3,Carre_RLE_Madagascar_2020,Forêts denses humides de moyenne altitude de l...,NT
1809,Carre_RLE_Madagascar_2020_4,Carre_RLE_Madagascar_2020,Forêts denses humides de haute altitude de l’E...,EN
1810,Carre_RLE_Madagascar_2020_5,Carre_RLE_Madagascar_2020,Forêts azonales de l’Est sur cuirasses ferrugi...,EN
1811,Carre_RLE_Madagascar_2020_6,Carre_RLE_Madagascar_2020,Forêts humides de l’Ouest,CR
1812,Carre_RLE_Madagascar_2020_7,Carre_RLE_Madagascar_2020,Forêts subhumides de l’Ouest,DD
1813,Carre_RLE_Madagascar_2020_8,Carre_RLE_Madagascar_2020,Forêts galeries ou rivulaires,DD
1814,Carre_RLE_Madagascar_2020_9,Carre_RLE_Madagascar_2020,Forêts de Tapia,EN
1815,Carre_RLE_Madagascar_2020_10,Carre_RLE_Madagascar_2020,Forêts sèches de l’Ouest,EN


In [79]:

xwalks=[((1,13,18), 'MT2.1'),
((2,3,4,5,6,7,8), 'T1.1'),
((9,10,11),'T1.2'),
((12,),'T5.2'),
((13,),'T3.1'),
((19,),'T3.4'),
((20,),'SF1.1'),
((33,34),'MFT1.2'),
((35,),'MFT1.1'),
((36,),'FM1.3'),
((29,30,31,32),'M1.1'),
((24,25,26,27,28),'M1.3'),
((23,),'TF1.3'),
((22,),'F2.2'),
((21,),'F1.1'),
((21,),'F1.2')]

This code will iterate through this list and assign the suggested EFG code:

In [83]:
ins = """
INSERT INTO rle.assessment_get_xwalk  
(eco_id,efg_code,level,membership,assigned_by) 
values (%s,%s,6,0.1,'JRFeP') ON CONFLICT DO NOTHING"""

conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)

for x in xwalks:
    for j in x[0]:
        cur.execute(ins, ('Carre_RLE_Madagascar_2020_'+str(j),x[1]))

cur.close()
conn.commit()
if conn is not None:
    conn.close()